# In-Class Coding Lab: Data Analysis with Pandas

In this lab, we will perform a data analysis on the **RMS Titanic** passenger list. The RMS Titanic is one of the most famous ocean liners in history. On April 15, 1912 it sank after colliding with an iceberg in the North Atlantic Ocean. To learn more, read here: https://en.wikipedia.org/wiki/RMS_Titanic 

Our goal today is to perform a data analysis on a subset of the passenger list. We're looking for insights as to which types of passengers did and didn't survive. Women? Children? 1st Class Passengers? 3rd class? Etc. 

I'm sure you've heard the expression often said during emergencies: "Women and Children first" Let's explore this data set and find out if that's true!

Before we begin you should read up on what each of the columns mean in the data dictionary. You can find this information on this page: https://www.kaggle.com/c/titanic/data 


## Loading the data set

First we load the dataset into a Pandas `DataFrame` variable. The `sample(10)` method takes a random sample of 10 passengers from the data set.

In [34]:
import pandas as pd
import numpy as np

# this turns off warning messages
import warnings
warnings.filterwarnings('ignore')

passengers = pd.read_csv('CCL-titanic.csv')
passengers.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
204,205,1,3,"Cohen, Mr. Gurshon ""Gus""",male,18.0,0,0,A/5 3540,8.0500,NaN,S
233,234,1,3,"Asplund, Miss. Lillian Gertrud",female,5.0,4,2,347077,31.3875,NaN,S
112,113,0,3,"Barton, Mr. David John",male,22.0,0,0,324669,8.0500,NaN,S
666,667,0,2,"Butler, Mr. Reginald Fenton",male,25.0,0,0,234686,13.0000,NaN,S
616,617,0,3,"Danbom, Mr. Ernst Gilbert",male,34.0,1,1,347080,14.4000,NaN,S
726,727,1,2,"Renouf, Mrs. Peter Henry (Lillian Jefferys)",female,30.0,3,0,31027,21.0000,NaN,S
169,170,0,3,"Ling, Mr. Lee",male,28.0,0,0,1601,56.4958,NaN,S
764,765,0,3,"Eklund, Mr. Hans Linus",male,16.0,0,0,347074,7.7750,NaN,S
141,142,1,3,"Nysten, Miss. Anna Sofia",female,22.0,0,0,347081,7.7500,NaN,S
492,493,0,1,"Molson, Mr. Harry Markland",male,55.0,0,0,113787,30.5000,C30,S


## How many survived?

One of the first things we should do is figure out how many of the passengers in this data set survived. Let's start with isolating just the `'Survivied'` column into a series:

In [35]:
passengers['Survived'].sample(10)

377    0
432    1
28     1
122    0
187    1
15     1
610    0
571    1
61     1
32     1
Name: Survived, dtype: int64

There's too many to display so we just display a random sample of 10 passengers. 

- 1 means the passenger survivied
- 0 means the passenger died

What we really want is to count the number of survivors and deaths. We do this by querying the `value_counts()` of the `['Survived']` column, which returns a `Series` of counts, like this:

In [36]:
passengers['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

Only 342 passengers survived, and 549 perished. Let's observe this same data as percentages of the whole. We do this by adding the `normalize=True` named argument to the `value_counts()` method.

In [37]:
passengers['Survived'].value_counts(normalize=True)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

**Just 38% of passengers in this dataset survived.**

### Now you Try it!

**FIRST** Write a Pandas expression to display counts of males and female passengers using the `Sex` variable:

In [38]:
passengers['Sex'].value_counts()


male      577
female    314
Name: Sex, dtype: int64

**NEXT** Write a Pandas expression to display male /female passenger counts as a percentage of the whole number of passengers in the data set.

In [39]:
passengers["Sex"].value_counts(normalize=True)


male      0.647587
female    0.352413
Name: Sex, dtype: float64

If you got things working, you now know that **35% of passengers were female**.

## Who survivies? Men or Women?

We now know that 35% of the passengers were female, and 65% we male. 

**The next think to think about is how do survivial rates affect these numbers? **

If the ratio is about the same for surviviors only, then we can conclude that your **Sex** did not play a role in your survival on the RMS Titanic. 

Let's find out.

In [40]:
survivors = passengers[passengers['Survived'] ==1]
survivors['PassengerId'].count()

342

Still **342** like we discovered originally. Now let's check the **Sex** split among survivors only:

In [41]:
survivors['Sex'].value_counts()

female    233
male      109
Name: Sex, dtype: int64

WOW! That is a huge difference! But you probably can't see it easily. Let's represent it in a `DataFrame`, so that it's easier to visualize:

In [42]:
sex_all_series = passengers['Sex'].value_counts()
sex_survivor_series = survivors['Sex'].value_counts()

sex_comparision_df = pd.DataFrame({ 'AllPassengers' : sex_all_series, 'Survivors' : sex_survivor_series })
sex_comparision_df['SexSurvivialRate'] = sex_comparision_df['Survivors'] / sex_comparision_df['AllPassengers']
sex_comparision_df

,AllPassengers,Survivors,SexSurvivialRate
female,314,233,0.742038
male,577,109,0.188908


 **So, females had a 74% survival rate. Much better than the overall rate of 38%**
 
We should probably briefly explain the code above. 

- The first two lines get a series count of all passengers by Sex (male / female) and count of survivors by sex
- The third line creates DataFrame. Recall a pandas dataframe is just a dict of series. We have two keys 'AllPassengers' and 'Survivors'
- The  fourth line creates a new column in the dataframe which is just the survivors / all passengers to get the rate of survival for that Sex.

## Feature Engineering: Adults and Children

Sometimes the variable we want to analyze is not readily available, but can be created from existing data. This is commonly referred to as **feature engineering**. The name comes from machine learning where we use data called *features* to predict an outcome. 

Let's create a new feature called `'AgeCat'` as follows:

- When **Age** <=18 then 'Child'
- When **Age** >18 then 'Adult'

This is easy to do in pandas. First we create the column and set all values to `np.nan` which means 'Not a number'. This is Pandas way of saying no value. Then we set the values based on the rules we set for the feature.

In [43]:
passengers['AgeCat'] = np.nan # Not a number
passengers['AgeCat'][ passengers['Age'] <=18 ] = 'Child'
passengers['AgeCat'][ passengers['Age'] > 18 ] = 'Adult'
passengers.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeCat
273,274,0,1,"Natsch, Mr. Charles H",male,37.0,0,1,PC 17596,29.7000,C118,C,Adult
520,521,1,1,"Perreault, Miss. Anne",female,30.0,0,0,12749,93.5000,B73,S,Adult
172,173,1,3,"Johnson, Miss. Eleanor Ileen",female,1.0,1,1,347742,11.1333,NaN,S,Child
683,684,0,3,"Goodwin, Mr. Charles Edward",male,14.0,5,2,CA 2144,46.9000,NaN,S,Child
792,793,0,3,"Sage, Miss. Stella Anna",female,NaN,8,2,CA. 2343,69.5500,NaN,S,NaN


Let's get the count and distrubutions of Adults and Children on the passenger list.

In [44]:
passengers['AgeCat'].value_counts()

Adult    575
Child    139
Name: AgeCat, dtype: int64

And here's the percentage as a whole:

In [45]:
passengers['AgeCat'].value_counts(normalize=True)

Adult    0.805322
Child    0.194678
Name: AgeCat, dtype: float64

So close to **80%** of the passengers were adults. Once again let's look at the ratio of `AgeCat` for survivors only. If your age has no bearing of survivial, then the rates should be the same. 

Here's the counts of Adult / Children among the survivors only:

In [46]:
survivors = passengers[passengers['Survived'] ==1]
survivors['AgeCat'].value_counts()

Adult    220
Child     70
Name: AgeCat, dtype: int64

### Now You Try it!

Calculate the `AgeCat` survival rate, similar to how we did for the `SexSurvivalRate`. 

In [47]:
agecat_all_series = passengers['AgeCat'].value_counts()
agecat_survivor_series = survivors['AgeCat'].value_counts()
agecat_survival_rate = passengers['Agecat'].value_counts(normalize=True)
sex_comparision_df = pd.DataFrame({ 'AllPassengers' : sex_all_series, 'Survivors' : sex_survivor_series, 'Passengers' : sex_all_series })
sex_comparision_df['SexSurvivialRate'] = sex_comparision_df['Survivors'] / sex_comparision_df['AllPassengers'] / sex_comparision_df['Passengers']
sex_comparision_df
# todo make a data frame, add AgeCatSurvivialRate column, display dataframe 


KeyError: 'Agecat'

**So, children had a 50% survival rate, better than the overall rate of 38%**

## So, women and children first?

It looks like the RMS really did have the motto: "Women and Children First."

Here's our insights. We know:

- If you were a passenger, you had a 38% chance of survival.
- If you were a female passenger, you had a 74% chance of survival.
- If you were a child passenger, you had a 50% chance of survival. 


### Now you try it for Passenger Class

Repeat this process for `Pclass` The passenger class variable. Display the survival rates for each passenger class. What does the information tell you about passenger class and survival rates?

I'll give you a hint... "Money Talks"


In [49]:
pc_all_series = passengers['Pclass'].value_counts()
pc_survivor_series = survivors['Pclass'].value_counts()
pc_comparision_df = pd.DataFrame({ 'AllPassengers' : pc_all_series, 'Survivors' : pc_survivor_series})
pc_comparision_df['SexSurvivalRate'] = pc_comparision_df['Survivors'] / pc_comparision_df['AllPassengers']
pc_comparision_df

,AllPassengers,Survivors,SexSurvivalRate
1,216,136,0.629630
2,184,87,0.472826
3,491,119,0.242363
